**Project** ---> <h4><b>TEXT CLASSIFICATION USING TENSORFLOW.</b></h4>
<br>
<h5>Let's start the project.</h5>

# **Importing libraries**

In [1]:
import numpy as np
#tensorflow packages
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# **Loading IMDb Reviews Dataset**



In [2]:
train_data, validation_data, test_data = tfds.load("imdb_reviews",
                                                   split=('train[:60%]', 'train[60%:]', 'test'),
                                                   as_supervised=True)

<h4>This line loads the IMDb Reviews dataset from TensorFlow Datasets and splits it into training, validation, and test sets. It uses a 60-20-20 split for training, validation, and test data respectively.</h4>

In [3]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

# **Checking GPU Availability**

<h4>This will check whether a GPU is available for TensorFlow to use or not.</h4>

In [4]:
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT available")

GPU is NOT available


In [5]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

<h5>So, after executing this line of code, <b>train_example_batch</b> will contain a batch of 10 training examples, and <b>train_labels_batch</b> will contain their corresponding labels. These batches of data are typically used for training machine learning models in batches to improve training efficiency.</h5>

In [6]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [7]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

# **Embedding & Defining TensorFlow Hub Layer**

<h5> This defines a TensorFlow Hub layer using a pre-trained model for text embedding. You'll need to prefetch data with the actual link to the pre-trained model you want to use.</h5>

In [8]:
Embedding = ("https://www.kaggle.com/models/google/gnews-swivel/frameworks/TensorFlow2/variations/tf2-preview-20dim/versions/1")

In [9]:
hub_layer = hub.KerasLayer(Embedding, output_shape=[20], input_shape = [], dtype = tf.string, trainable=True)

In [10]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

# **Now it's time for Building The Model**



In [11]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

<h5>Here, we have build a sequential model using Keras.
<br>This model consists of the TensorFlow Hub layer, followed by a densely connected layer with 16 units and ReLU activation, and finally a densely connected output layer with 1 unit.</h5>

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# **Compiling the Model**

<h5>This will compile the model, specifying the optimizer, loss function (Binary Crossentropy for binary classification), and evaluation metrics (accuracy).</h5>

In [13]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits= True),
              metrics=['accuracy'])

# **Train the Model**

<h5>This trains the model using the training data, shuffling the data with a buffer size of 10000, batching it into batches of 100 examples, and training it for 25 epochs. It also validates the model's performance on the validation data.</h5>


In [14]:
history = model.fit(train_data.shuffle(10000).batch(100), epochs = 25,
                    validation_data = validation_data.batch(100),
                    verbose =1)

Epoch 1/25



150/150 [==============================] - 3s 15ms/step - loss: 0.6842 - accuracy: 0.5495 - val_loss: 0.5933 - val_accuracy: 0.6476
Epoch 2/25
150/150 [==============================] - 2s 13ms/step - loss: 0.5288 - accuracy: 0.7207 - val_loss: 0.4706 - val_accuracy: 0.7688
Epoch 3/25
150/150 [==============================] - 2s 13ms/step - loss: 0.4072 - accuracy: 0.8105 - val_loss: 0.3929 - val_accuracy: 0.8109
Epoch 4/25
150/150 [==============================] - 2s 13ms/step - loss: 0.3283 - accuracy: 0.8593 - val_loss: 0.3479 - val_accuracy: 0.8429
Epoch 5/25
150/150 [==============================] - 2s 13ms/step - loss: 0.2779 - accuracy: 0.8834 - val_loss: 0.3245 - val_accuracy: 0.8533
Epoch 6/25
150/150 [==============================] - 2s 13ms/step - loss: 0.2413 - accuracy: 0.9015 - val_loss: 0.3112 - val_accuracy: 0.8640
Epoch 7/25
150/150 [==============================] - 2s 14ms/step - loss: 0.2077 - accuracy: 0.9169 - val_loss: 0.3108 - val_accuracy: 0.8738
Epoch 8/25

# **Evaluate the Model**

In [15]:
results = model.evaluate(test_data.batch(100), verbose=2)
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name,value))

250/250 - 2s - loss: 0.6651 - accuracy: 0.8394 - 2s/epoch - 7ms/step
loss: 0.665
accuracy: 0.839


<h5>Finally, this code evaluates the trained model on the test data, printing out the evaluation metrics (loss and accuracy) for the test set.</h5>
<b>Loss is 67% & accuracy is 84%</b>

<h2 style="text-align: center;">!!! Happy  Coding !!!</h2>